In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_excel("3.xlsx")

In [5]:

df = df.dropna(subset=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])

In [6]:
num_cols = df.shape[1]
min_non_null = int(0.3 * num_cols)
df = df.dropna(thresh=min_non_null)


In [7]:

df_internal_validation = df[df['Patient_Hospital']!="Hospital2"]
df_external_validation = df[df['Patient_Hospital']=="Hospital2"]
df_internal_validation = df_internal_validation.drop(columns=['Patient_Hospital'])
df_external_validation = df_external_validation.drop(columns=['Patient_Hospital'])

In [8]:
X = df_internal_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y  = df_internal_validation[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]

In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0)
df_imputed = imputer.fit_transform(X)

y_ex = pd.DataFrame(y.loc[ : ,'Outcome_InhospitalMortality'])


d:\python\Lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [10]:
y_train = y[['Outcome_InhospitalMortality']]

In [11]:
X_ex = df_external_validation.drop(columns=['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis'])
y_ex  = df_external_validation[['Outcome_InhospitalMortality', 'TM_S_Intubation', 'Outcome_ICUadmission','TM_S_Dialysis']]

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


imputer = IterativeImputer(max_iter=10, random_state=0)
df_imputed_ex = imputer.fit_transform(X_ex)

y_ex = pd.DataFrame(y_ex.loc[ : ,'Outcome_InhospitalMortality'])


In [12]:
X_train , X_test , y_train , y_test = train_test_split(df_imputed,y, test_size = 0.3 , random_state = 42)

In [13]:
y_train = y_train['Outcome_InhospitalMortality']
y_test= y_test['Outcome_InhospitalMortality']

In [14]:
arr_ex =df_imputed_ex #.to_numpy()

from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_ex_normalize = transform.fit_transform(arr_ex)
df_ex_normalize =pd.DataFrame(df_ex_normalize)
df_ex_normalize.columns = X_ex.columns.tolist()

X_ex = df_ex_normalize

In [15]:
arr_tain =X_train#.to_numpy()
from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_normalize = transform.fit_transform(arr_tain)
df_normalize =pd.DataFrame(df_normalize)
df_normalize.columns = X.columns.tolist()
X_train = df_normalize

In [16]:
arr =X_test#.to_numpy()
from sklearn import preprocessing
transform = preprocessing.StandardScaler()
df_test_normalize = transform.fit_transform(arr)
df_test_normalize =pd.DataFrame(df_test_normalize)
df_test_normalize.columns = X.columns.tolist()
X_test =df_test_normalize

In [17]:
from sklearn.linear_model import Lasso

def lasso_MO_ICU(X_train, y_train,X_test , X_ex):

    
    lasso = Lasso(alpha=0.0001,random_state=42 )
    lasso.fit(X_train, y_train)


    absolute_coeffs = np.abs(lasso.coef_)
    sorted_indices = np.argsort(absolute_coeffs)[::-1]

    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    X_ex = pd.DataFrame(X_ex)

    selected_feature_indices = sorted_indices[:40]
    selected_feature_indices = np.where(lasso.coef_ != 0)[0]


    X_train_selected_Mortality_ICU = X_train.iloc[:, selected_feature_indices]
    X_test_selected_Mortality_ICU = X_test.iloc[:, selected_feature_indices]
    X_ex = X_ex.iloc[:, selected_feature_indices]
    return X_train_selected_Mortality_ICU ,X_test_selected_Mortality_ICU,X_ex

X_train,X_test,X_ex = lasso_MO_ICU(X_train, y_train,X_test,X_ex)


d:\python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.532e-01, tolerance: 8.537e-02
  model = cd_fast.enet_coordinate_descent(


In [18]:
X_train = X_train.drop(columns=['L1_BloodGroup_First'])
X_test =X_test.drop(columns=['L1_BloodGroup_First'])
X_ex =X_ex.drop(columns=['L1_BloodGroup_First'])

In [19]:
from imblearn.under_sampling import TomekLinks , RandomUnderSampler

def balancing_dataset(X_train,y_train):
    
    tl = RandomUnderSampler(random_state=42)
    X_resampled_train , y_resampled_train = tl.fit_resample(X_train,y_train)
    X_resampled_test , y_resampled_test = tl.fit_resample(X_test,y_test)
    
    return (X_resampled_train, y_resampled_train,X_resampled_test , y_resampled_test)
a,b, X_test , y_test = balancing_dataset(X_train,y_train)

In [20]:
X_train,y_train = a,b 

In [21]:

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [22]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [23]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score
)
def logistic_regression_classifier(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_lr = {'random_state':[42]}


    lr = LogisticRegression(random_state=42)


    grid_search_lr = GridSearchCV(
        estimator=lr,
        param_grid=parameters_lr,
        ##cv=5,
        n_jobs=-1
    )

    logreg_cv = grid_search_lr.fit(X_train, y_train)

    
    best_classifier_lr = logreg_cv.best_estimator_

 
    y_predicted_Mortality_lr = best_classifier_lr.predict(X_test)

 
    accuracy = accuracy_score(y_test, y_predicted_Mortality_lr)
    cm = confusion_matrix(y_test, y_predicted_Mortality_lr)
    precision = precision_score(y_test, y_predicted_Mortality_lr)
    recall = recall_score(y_test, y_predicted_Mortality_lr)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_lr)


    Logistic_Regression = {}
    Logistic_Regression["accuracy"] = accuracy
    Logistic_Regression["precision"] = precision
    Logistic_Regression["recall"] = recall
    Logistic_Regression["specificity"] = specificity
    Logistic_Regression["f1"] = f1
    Logistic_Regression["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_lr)

    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_predicted_Mortality_lr
    
    y_predicted_Mortality_boost = best_classifier_lr.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    Logistic_Regression["accuracy_ex"] = accuracy
    Logistic_Regression["precision_ex"] = precision
    Logistic_Regression["recall_ex"] = recall
    Logistic_Regression["specificity_ex"] = specificity
    Logistic_Regression["f1_ex"] = f1
    Logistic_Regression["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
   
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost
    
    Logistic_Regression = (Logistic_Regression, y_plot)
    
    return Logistic_Regression


In [24]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_svm(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    svm_classifier = svm.SVC(random_state=42)


    parameters_svm = {'random_state':[42]}


    grid_search_svm = GridSearchCV(
        estimator=svm_classifier,
        param_grid=parameters_svm,
        #cv=5
    )
    
   
    svm_cv = grid_search_svm.fit(X_train, y_train)

 
    best_classifier_svm = svm_cv.best_estimator_


    y_predicted_Mortality_svm = best_classifier_svm.predict(X_test)

   
    accuracy = accuracy_score(y_test, y_predicted_Mortality_svm)
    cm = confusion_matrix(y_test, y_predicted_Mortality_svm)
    precision = precision_score(y_test, y_predicted_Mortality_svm)
    recall = recall_score(y_test, y_predicted_Mortality_svm)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_svm)

    svm_results = {}
    svm_results["accuracy"] = accuracy
    svm_results["precision"] = precision
    svm_results["recall"] = recall
    svm_results["specificity"] = specificity
    svm_results["f1"] = f1
    svm_results["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_svm)

  
    y_plot = {}
    y_plot["y_true"] = y_test  
    y_plot["y_predicted"] = y_predicted_Mortality_svm

    y_predicted_Mortality_boost = best_classifier_svm.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    svm_results["accuracy_ex"] = accuracy
    svm_results["precision_ex"] = precision
    svm_results["recall_ex"] = recall
    svm_results["specificity_ex"] = specificity
    svm_results["f1_ex"] = f1
    svm_results["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
   
    y_plot["y_true_ex"] =y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost
    
    svm_results = (svm_results, y_plot)
    
    return svm_results



In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_tree(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_tree = {"random_state":[42]}

 
    tree = DecisionTreeClassifier(random_state=42)

 
    grid_search_tree = GridSearchCV(
        estimator=tree,
        param_grid=parameters_tree,
        #cv=5
    )
    

    tree_cv = grid_search_tree.fit(X_train, y_train)

  
    best_classifier_tree = tree_cv.best_estimator_

 
    y_predicted_Mortality_tree = best_classifier_tree.predict(X_test)

 
    accuracy = accuracy_score(y_test, y_predicted_Mortality_tree)
    cm = confusion_matrix(y_test, y_predicted_Mortality_tree)
    precision = precision_score(y_test, y_predicted_Mortality_tree)
    recall = recall_score(y_test, y_predicted_Mortality_tree)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_tree)

  
    tree_results = {}
    tree_results["accuracy"] = accuracy
    tree_results["precision"] = precision
    tree_results["recall"] = recall
    tree_results["specificity"] = specificity
    tree_results["f1"] = f1
    tree_results["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_tree)


    y_plot = {}
    y_plot["y_true"] = y_test 
    y_plot["y_predicted"] = y_predicted_Mortality_tree
    
    y_predicted_Mortality_boost = best_classifier_tree.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    tree_results["accuracy_ex"] = accuracy
    tree_results["precision_ex"] = precision
    tree_results["recall_ex"] = recall
    tree_results["specificity_ex"] = specificity
    tree_results["f1_ex"] = f1
    tree_results["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
   
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost

    tree_results = (tree_results, y_plot)
    
    return tree_results



In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_knn(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_knn = {}

 
    knn = KNeighborsClassifier()


    grid_search_knn = GridSearchCV(
        estimator=knn,
        param_grid=parameters_knn,
        #cv=5
    )
    

    knn_cv = grid_search_knn.fit(X_train, y_train)

    best_classifier_knn = knn_cv.best_estimator_

    y_predicted_Mortality_knn = best_classifier_knn.predict(X_test)

    accuracy = accuracy_score(y_test, y_predicted_Mortality_knn)
    cm = confusion_matrix(y_test, y_predicted_Mortality_knn)
    precision = precision_score(y_test, y_predicted_Mortality_knn)
    recall = recall_score(y_test, y_predicted_Mortality_knn)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_knn)

    knn_results = {}
    knn_results["accuracy"] = accuracy
    knn_results["precision"] = precision
    knn_results["recall"] = recall
    knn_results["specificity"] = specificity
    knn_results["f1"] = f1
    knn_results["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_knn)


    y_plot = {}
    y_plot["y_true"] = y_test  
    y_plot["y_predicted"] = y_predicted_Mortality_knn
    
    y_predicted_Mortality_boost = best_classifier_knn.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    knn_results["accuracy_ex"] = accuracy
    knn_results["precision_ex"] = precision
    knn_results["recall_ex"] = recall
    knn_results["specificity_ex"] = specificity
    knn_results["f1_ex"] = f1
    knn_results["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
   
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost

    knn_results = (knn_results, y_plot)
    
    return knn_results



In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_forest(X_train, y_train, X_test, y_test , X_ex ,y_ex):
    parameters_forest = {'random_state':[42]}


    forest = RandomForestClassifier(random_state=42)


    grid_search_forest = GridSearchCV(
        estimator=forest,
        param_grid=parameters_forest,
        #cv=5
    )
    

    forest_cv = grid_search_forest.fit(X_train, y_train)

    best_classifier_forest = forest_cv.best_estimator_

    y_predicted_Mortality_forest = best_classifier_forest.predict(X_test)


    accuracy = accuracy_score(y_test, y_predicted_Mortality_forest)
    cm = confusion_matrix(y_test, y_predicted_Mortality_forest)
    precision = precision_score(y_test, y_predicted_Mortality_forest)
    recall = recall_score(y_test, y_predicted_Mortality_forest)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_forest)


    forest_results = {}
    forest_results["accuracy"] = accuracy
    forest_results["precision"] = precision
    forest_results["recall"] = recall
    forest_results["specificity"] = specificity
    forest_results["f1"] = f1
    forest_results["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_forest)

    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_predicted_Mortality_forest
    
    y_predicted_Mortality_boost = best_classifier_forest.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    forest_results["accuracy_ex"] = accuracy
    forest_results["precision_ex"] = precision
    forest_results["recall_ex"] = recall
    forest_results["specificity_ex"] = specificity
    forest_results["f1_ex"] = f1
    forest_results["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
   
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost

    return forest_results, y_plot


In [28]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_neural(X_train, y_train, X_test, y_test , X_ex ,y_ex):

    parameters_neural = {
        'hidden_layer_sizes': [(100,), (50, 50)],  # You can adjust the architecture here
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.0001],
        'max_iter': [200],
        'random_state': [42],
        'early_stopping': [True],
        'validation_fraction': [0.1],
        'n_iter_no_change': [10]
    }


    neural = MLPClassifier(random_state=42)

    grid_search_neural = GridSearchCV(
        estimator=neural,
        param_grid=parameters_neural,
        #cv=5
    )
    

    neural_cv = grid_search_neural.fit(X_train, y_train)

   
    best_classifier_neural = neural_cv.best_estimator_

    
    y_predicted_Mortality_neural = best_classifier_neural.predict(X_test)

    accuracy = accuracy_score(y_test, y_predicted_Mortality_neural)
    cm = confusion_matrix(y_test, y_predicted_Mortality_neural)
    precision = precision_score(y_test, y_predicted_Mortality_neural)
    recall = recall_score(y_test, y_predicted_Mortality_neural)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_neural)

 
    neural_results = {}
    neural_results["accuracy"] = accuracy
    neural_results["precision"] = precision
    neural_results["recall"] = recall
    neural_results["specificity"] = specificity
    neural_results["f1"] = f1
    neural_results["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_neural)

    
    y_plot = {}
    y_plot["y_true"] = y_test  
    y_plot["y_predicted"] = y_predicted_Mortality_neural
    
    y_predicted_Mortality_boost = best_classifier_neural.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    neural_results["accuracy_ex"] = accuracy
    neural_results["precision_ex"] = precision
    neural_results["recall_ex"] = recall
    neural_results["specificity_ex"] = specificity
    neural_results["f1_ex"] = f1
    neural_results["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
   
    y_plot["y_true_ex"] =y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost

    return neural_results, y_plot



In [29]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

def train_and_evaluate_boost(X_train, y_train, X_test, y_test , X_ex ,y_ex):
  
    parameters_boost = {'random_state':[42] }

 
    boost = XGBClassifier(random_state=42)


    grid_search_boost = GridSearchCV(
        estimator=boost,
        param_grid=parameters_boost,
        #cv=5
    )
    

    boost_cv = grid_search_boost.fit(X_train, y_train)


    best_classifier_boost = boost_cv.best_estimator_


    y_predicted_Mortality_boost = best_classifier_boost.predict(X_test)


    accuracy = accuracy_score(y_test, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_test, y_predicted_Mortality_boost)
    precision = precision_score(y_test, y_predicted_Mortality_boost)
    recall = recall_score(y_test, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_test, y_predicted_Mortality_boost)

  
    boost_results = {}
    boost_results["accuracy"] = accuracy
    boost_results["precision"] = precision
    boost_results["recall"] = recall
    boost_results["specificity"] = specificity
    boost_results["f1"] = f1
    boost_results["AUC"] = roc_auc_score(y_test, y_predicted_Mortality_boost)
    y_plot = {}
    y_plot["y_true"] = y_test
    y_plot["y_predicted"] = y_predicted_Mortality_boost
    
    y_predicted_Mortality_boost = best_classifier_boost.predict(X_ex)
    accuracy = accuracy_score(y_ex, y_predicted_Mortality_boost)
    cm = confusion_matrix(y_ex, y_predicted_Mortality_boost)
    precision = precision_score(y_ex, y_predicted_Mortality_boost)
    recall = recall_score(y_ex, y_predicted_Mortality_boost)
    specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    f1 = f1_score(y_ex, y_predicted_Mortality_boost)

  
    
    boost_results["accuracy_ex"] = accuracy
    boost_results["precision_ex"] = precision
    boost_results["recall_ex"] = recall
    boost_results["specificity_ex"] = specificity
    boost_results["f1_ex"] = f1
    boost_results["AUC_ex"] = roc_auc_score(y_ex, y_predicted_Mortality_boost)
    y_plot["y_true_ex"] = y_ex
    y_plot["y_predicted_ex"] = y_predicted_Mortality_boost
    
    return boost_results, y_plot



In [30]:
list_Outcome_InhospitalMortality=[]

y_train_ = pd.DataFrame(y_train)
y_test_ = pd.DataFrame(y_test)



In [31]:
y_train_ = np.array(y_train)
y_test_ = np.array(y_test)

In [32]:

X_test=np.array(X_test)
y_ex_ = np.array(y_ex['Outcome_InhospitalMortality'].tolist())
X_ex = np.array(X_ex)



In [33]:
y_ex_

array([1., 1., 1., ..., 1., 1., 1.])

In [34]:

list_Outcome_InhospitalMortality.extend([logistic_regression_classifier(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                            train_and_evaluate_svm(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_tree(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_knn(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_forest(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_neural(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_),
                                        train_and_evaluate_boost(X_train, y_train_, X_test, y_test_, X_ex ,y_ex_)])


result_dic_list_Outcome_InhospitalMortality = dict(zip(['logistic_regression', 'svm', 'tree', 'knn', 'forest', 'neural', 'boost'], list_Outcome_InhospitalMortality))





merged_dict = {}

# List of dictionary names and their corresponding dictionaries
dict_list = [('Outcome_InhospitalMortality', result_dic_list_Outcome_InhospitalMortality)]

# Merge the dictionaries
for name, result_dict in dict_list:
    merged_dict[name] = result_dict

# The merged_dict now contains all the dictionaries merged together


df = pd.DataFrame(merged_dict)


df = df.transpose()


df.reset_index(inplace=True)
df.rename(columns={'index': 'Method'}, inplace=True)





data = []

for outcome, models in merged_dict.items():
    for model, metrics in models.items():
        accuracy, precision, recall, specificity, f1, AUC,accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex, AUC_ex = metrics[0]['accuracy'], metrics[0]['precision'], metrics[0]['recall'], metrics[0]['specificity'], metrics[0]['f1'], metrics[0]['AUC'],metrics[0]['accuracy_ex'], metrics[0]['precision_ex'], metrics[0]['recall_ex'], metrics[0]['specificity_ex'], metrics[0]['f1_ex'], metrics[0]['AUC_ex']
        y_true, y_predicted,y_true_ex,y_predicted_ex = metrics[1]['y_true'], metrics[1]['y_predicted'],metrics[1]['y_true_ex'], metrics[1]['y_predicted_ex']
        data.append([outcome, model,accuracy, precision, recall, specificity, f1, AUC,accuracy_ex, precision_ex, recall_ex, specificity_ex, f1_ex, AUC_ex, y_true.tolist(), y_predicted.tolist(),y_true_ex.tolist(),y_predicted_ex.tolist()])

columns = ['Outcome', 'Model', 'Accuracy', 'Precision', 'Recall', 'Specificity', 'F1','AUC', 'Accuracy_ex', 'Precision_ex', 'Recall_ex', 'Specificity_ex', 'F1_ex','AUC_ex', 'y_true', 'y_predicted', 'y_true_ex', 'y_predicted_ex']

df = pd.DataFrame(data, columns=columns)



In [35]:
df

,Outcome,Model,Accuracy,Precision,Recall,Specificity,F1,AUC,Accuracy_ex,Precision_ex,Recall_ex,Specificity_ex,F1_ex,AUC_ex,y_true,y_predicted,y_true_ex,y_predicted_ex
0,Outcome_InhospitalMortality,logistic_regression,0.699143,0.695378,0.708779,0.689507,0.702015,0.699143,0.728176,0.897106,0.744662,0.663158,0.813807,0.703910,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,Outcome_InhospitalMortality,svm,0.719486,0.738928,0.678801,0.760171,0.707589,0.719486,0.704045,0.915394,0.693060,0.747368,0.788861,0.720214,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ..."
2,Outcome_InhospitalMortality,tree,0.613490,0.597070,0.698073,0.528908,0.643633,0.613490,0.614620,0.842167,0.636121,0.529825,0.724785,0.582973,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
3,Outcome_InhospitalMortality,knn,0.641328,0.633603,0.670236,0.612420,0.651405,0.641328,0.687722,0.874179,0.710854,0.596491,0.784102,0.653673,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ..."
4,Outcome_InhospitalMortality,forest,0.759101,0.805556,0.683084,0.835118,0.739282,0.759101,0.704755,0.913551,0.695730,0.740351,0.789899,0.718040,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, ..."
5,Outcome_InhospitalMortality,neural,0.713062,0.715835,0.706638,0.719486,0.711207,0.713062,0.714691,0.899336,0.723310,0.680702,0.801775,0.702006,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, ..."
6,Outcome_InhospitalMortality,boost,0.743041,0.772182,0.689507,0.796574,0.728507,0.743041,0.684173,0.896149,0.683274,0.687719,0.775366,0.685497,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, ..."


In [36]:
df.to_excel('CML.xlsx',index=False)